In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score

class Conv2d:
    def __init__(self, F, C, FH, FW, P, S, initializer=None, optimizer=None, activation=None):
        self.F = F  # Number of filters
        self.C = C  # Number of input channels
        self.FH = FH # Filter height
        self.FW = FW # Filter width
        self.P = P  # Padding size
        self.S = S  # Stride size
        self.initializer = initializer
        self.optimizer = optimizer
        self.activation = activation
        self.W = self.initializer.W(F, C, FH, FW) if initializer else np.random.randn(F, C, FH, FW) * 0.01
        self.B = self.initializer.B(F) if initializer else np.zeros(F)
        self.dW = np.zeros_like(self.W)
        self.dB = np.zeros_like(self.B)
        self.X = None

    def output_shape2d(self, H, W):
        OH = (H + 2 * self.P - self.FH) // self.S + 1
        OW = (W + 2 * self.P - self.FW) // self.S + 1
        return OH, OW

    def forward(self, X):
        self.X = X
        N, C, H, W = X.shape
        OH, OW = self.output_shape2d(H, W)
        A = np.zeros((N, self.F, OH, OW))
        X_padded = np.pad(X, ((0, 0), (0, 0), (self.P, self.P), (self.P, self.P)), 'constant')

        for n in range(N):
            for f in range(self.F):
                for oh in range(OH):
                    for ow in range(OW):
                        h_start = oh * self.S
                        h_end = h_start + self.FH
                        w_start = ow * self.S
                        w_end = w_start + self.FW
                        A[n, f, oh, ow] = np.sum(X_padded[n, :, h_start:h_end, w_start:w_end] * self.W[f, :, :, :]) + self.B[f]

        return self.activation.forward(A) if self.activation else A

    def backward(self, dA):
        if self.activation:
            dA = self.activation.backward(dA)

        N, F, OH, OW = dA.shape
        C = self.C
        H, W = self.X.shape[2:]
        FH, FW = self.FH, self.FW
        P, S = self.P, self.S

        dX = np.zeros_like(self.X)
        dW = np.zeros_like(self.W)
        dB = np.zeros_like(self.B)
        X_padded = np.pad(self.X, ((0, 0), (0, 0), (P, P), (P, P)), 'constant')
        dX_padded = np.pad(dX, ((0, 0), (0, 0), (P, P), (P, P)), 'constant')

        for n in range(N):
            for f in range(F):
                for c in range(C):
                    for fh in range(FH):
                        for fw in range(FW):
                            for oh in range(OH):
                                for ow in range(OW):
                                    dX_padded[n, c, oh * S + fh, ow * S + fw] += dA[n, f, oh, ow] * self.W[f, c, fh, fw]
                                    dW[f, c, fh, fw] += dA[n, f, oh, ow] * X_padded[n, c, oh * S + fh, ow * S + fw]
                dB[f] += np.sum(dA[n, f, :, :])

        if P > 0:
            dX = dX_padded[:, :, P:-P, P:-P]
        else:
            dX = dX_padded

        self.dW = dW / N
        self.dB = dB / N

        if self.optimizer:
            self.W = self.optimizer.update(self.W, self.dW)
            self.B = self.optimizer.update(self.B, self.dB)

        return dX

In [ ]:
#flowing CNN2 forwards
x = np.array([[[[ 1,  2,  3,  4],
                [ 5,  6,  7,  8],
                [ 9, 10, 11, 12],
                [13, 14, 15, 16]]]])

#
w = np.array([[[[ 0.,  0.,  0.],
               [ 0.,  1.,  0.],
               [ 0., -1.,  0.]]],

              [[[ 0.,  0.,  0.],
               [ 0., -1.,  1.],
               [ 0.,  0.,  0.]]]])

#
class SimpleInitializerConv2d:
    def __init__(self, sigma=0.01):
        self.sigma = sigma
    def W(self, F, C, FH, FW):
        return self.sigma * np.random.randn(F,C,FH,FW)
    def B(self, F):
        return np.zeros(F)

# Conv2d layer
conv2d = Conv2d(F=2, C=1, FH=3, FW=3, P=0, S=1, initializer=SimpleInitializerConv2d())
conv2d.W = w 

# Forward propagation
output_forward = conv2d.forward(x)
print("Forward Propagation Output:")
print(output_forward)

# Backpropagation
delta = np.array([[[[ -4,  -4],
                   [ 10,  11]]],

                  [[[  1,  -7],
                   [  1, -11]]]])

# 
delta_reshaped = delta.reshape(1, 2, 2, 2)

output_backward = conv2d.backward(delta_reshaped)
print("\nBackward Propagation Output (dX):")
print(output_backward)
print("\nBackward Propagation dW:")
print(conv2d.dW)
print("\nBackward Propagation dB:")
print(conv2d.dB)

In [ ]:
def calculate_output_size(input_height, input_width, padding, filter_height, filter_width, stride):
    output_height = (input_height + 2 * padding - filter_height) // stride + 1
    output_width = (input_width + 2 * padding - filter_width) // stride + 1
    return int(output_height), int(output_width)

#######
input_h, input_w = 6, 6
padding_h, padding_w = 0, 0
filter_h, filter_w = 3, 3
stride_h, stride_w = 1, 1

output_h, output_w = calculate_output_size(input_h, input_w, padding_h, filter_h, filter_w, stride_h)
print(f"Output size for input ({input_h}x{input_w}), padding {padding_h}, filter ({filter_h}x{filter_w}), stride {stride_h}: ({output_h}x{output_w})")

In [ ]:
class MaxPool2D:
    def __init__(self, pool_size, stride=None):
        self.pool_size = pool_size
        self.stride = stride if stride is not None else pool_size
        self.X = None
        self.arg_max = None

    def forward(self, X):
        self.X = X
        N, C, H, W = X.shape
        PH, PW = self.pool_size
        SH, SW = self.stride
        OH = (H - PH) // SH + 1
        OW = (W - PW) // SW + 1
        out = np.zeros((N, C, OH, OW))
        arg_max = np.zeros((N, C, OH, OW, 2), dtype=int)

        for n in range(N):
            for c in range(C):
                for oh in range(OH):
                    for ow in range(OW):
                        h_start = oh * SH
                        h_end = h_start + PH
                        w_start = ow * SW
                        w_end = w_start + PW
                        pool_window = X[n, c, h_start:h_end, w_start:w_end]
                        max_val = np.max(pool_window)
                        out[n, c, oh, ow] = max_val
                        arg_max_index = np.unravel_index(np.argmax(pool_window), pool_window.shape)
                        arg_max[n, c, oh, ow] = (h_start + arg_max_index[0], w_start + arg_max_index[1])

        self.arg_max = arg_max
        return out

    def backward(self, dout):
        dX = np.zeros_like(self.X)
        N, C, OH, OW = dout.shape
        PH, PW = self.pool_size
        SH, SW = self.stride

        for n in range(N):
            for c in range(C):
                for oh in range(OH):
                    for ow in range(OW):
                        h_idx, w_idx = self.arg_max[n, c, oh, ow]
                        dX[n, c, h_idx, w_idx] += dout[n, c, oh, ow]

        return dX

In [ ]:
class AveragePool2D:
    def __init__(self, pool_size, stride=None):
        self.pool_size = pool_size
        self.stride = stride if stride is not None else pool_size
        self.X = None

    def forward(self, X):
        self.X = X
        N, C, H, W = X.shape
        PH, PW = self.pool_size
        SH, SW = self.stride
        OH = (H - PH) // SH + 1
        OW = (W - PW) // SW + 1
        out = np.zeros((N, C, OH, OW))

        for n in range(N):
            for c in range(C):
                for oh in range(OH):
                    for ow in range(OW):
                        h_start = oh * SH
                        h_end = h_start + PH
                        w_start = ow * SW
                        w_end = ow * SW + PW
                        pool_window = X[n, c, h_start:h_end, w_start:w_end]
                        avg_val = np.mean(pool_window)
                        out[n, c, oh, ow] = avg_val
        return out

    def backward(self, dout):
        dX = np.zeros_like(self.X)
        N, C, OH, OW = dout.shape
        PH, PW = self.pool_size
        SH, SW = self.stride
        pool_area = PH * PW

        for n in range(N):
            for c in range(C):
                for oh in range(OH):
                    for ow in range(OW):
                        h_start = oh * SH
                        h_end = h_start + PH
                        w_start = ow * SW
                        w_end = ow * SW + PW
                        dX[n, c, h_start:h_end, w_start:w_end] += dout[n, c, oh, ow] / pool_area
        return dX

In [ ]:
class Flatten:
    def __init__(self):
        self.shape = None

    def forward(self, X):
        self.shape = X.shape
        return X.reshape(len(X), -1)

    def backward(self, dout):
        return dout.reshape(self.shape)

In [ ]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score

# 
class GetMiniBatch:
    def __init__(self, X, y, batch_size=20, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self._X = X[shuffle_index]
        self._y = y[shuffle_index]
        self._stop = (X.shape[0] + self.batch_size - 1) // self.batch_size  

    def __len__(self):
        return self._stop

    def __getitem__(self, item):
        if item >= self._stop:
            raise IndexError("Mini-batch index out of range")
        p0 = item * self.batch_size
        p1 = min(p0 + self.batch_size, len(self._X))
        return self._X[p0:p1], self._y[p0:p1]


class ReLU:
    def forward(self, A):
        self.A = A
        return np.maximum(0, A)

    def backward(self, dZ):
        return dZ * (self.A > 0)

class Softmax:
    def forward(self, X):
        self.Z = np.exp(X - np.max(X, axis=1, keepdims=True))
        return self.Z / np.sum(self.Z, axis=1, keepdims=True)

    def backward(self, dZ):
        dA = self.Z * dZ
        s = np.sum(dA, axis=1, keepdims=True)
        dA -= self.Z * s
        return dA

class FC:
    def __init__(self, n_in, n_out, initializer, optimizer, activation):
        self.W = initializer.W(n_in, n_out)
        self.B = initializer.B(n_out)
        self.optimizer = optimizer
        self.activation = activation

    def forward(self, X):
        self.X = X
        self.Z = np.dot(X, self.W) + self.B
        return self.activation.forward(self.Z)

    def backward(self, dA):
        dZ = self.activation.backward(dA)
        self.dW = np.dot(self.X.T, dZ)
        self.dB = np.sum(dZ, axis=0)
        dX = np.dot(dZ, self.W.T)
        self.optimizer.update(self)
        return dX

class SGD:
    def __init__(self, lr):
        self.lr = lr

    def update(self, layer):
        layer.W -= self.lr * layer.dW
        layer.B -= self.lr * layer.dB

class HeInitializer:
    def W(self, *args):
        if len(args) == 2:
            n_in, n_out = args
            return np.random.randn(n_in, n_out) * np.sqrt(2 / n_in)
        elif len(args) == 4:
            F, C, FH, FW = args
            return np.random.randn(F, C, FH, FW) * np.sqrt(2 / (C * FH * FW))
        else:
            raise ValueError("Invalid shape")

    def B(self, n_out):
        return np.zeros(n_out)

class Flatten:
    def __init__(self):
        self.shape = None

    def forward(self, X):
        self.shape = X.shape
        return X.reshape(len(X), -1)

    def backward(self, dout):
        return dout.reshape(self.shape)

class Conv2d:
    def __init__(self, F, C, FH, FW, P, S, initializer=None, optimizer=None, activation=None):
        self.F, self.C, self.FH, self.FW, self.P, self.S = F, C, FH, FW, P, S
        self.initializer = initializer
        self.optimizer = optimizer
        self.activation = activation
        self.W = initializer.W(F, C, FH, FW)
        self.B = initializer.B(F)

    def output_shape2d(self, H, W):
        OH = (H + 2 * self.P - self.FH) // self.S + 1
        OW = (W + 2 * self.P - self.FW) // self.S + 1
        return OH, OW

    def forward(self, X):
        self.X = X
        N, C, H, W = X.shape
        OH, OW = self.output_shape2d(H, W)
        A = np.zeros((N, self.F, OH, OW))
        X_pad = np.pad(X, ((0, 0), (0, 0), (self.P, self.P), (self.P, self.P)), 'constant')

        for n in range(N):
            for f in range(self.F):
                for oh in range(OH):
                    for ow in range(OW):
                        h0 = oh * self.S
                        w0 = ow * self.S
                        window = X_pad[n, :, h0:h0+self.FH, w0:w0+self.FW]
                        A[n, f, oh, ow] = np.sum(window * self.W[f]) + self.B[f]

        return self.activation.forward(A) if self.activation else A

    def backward(self, dA):
        if self.activation:
            dA = self.activation.backward(dA)

        N, F, OH, OW = dA.shape
        _, _, H, W = self.X.shape
        FH, FW, P, S = self.FH, self.FW, self.P, self.S
        dX = np.zeros_like(self.X)
        dW = np.zeros_like(self.W)
        dB = np.zeros_like(self.B)

        X_pad = np.pad(self.X, ((0,0), (0,0), (P,P), (P,P)), 'constant')
        dX_pad = np.pad(dX, ((0,0), (0,0), (P,P), (P,P)), 'constant')

        for n in range(N):
            for f in range(F):
                for oh in range(OH):
                    for ow in range(OW):
                        h0 = oh * S
                        w0 = ow * S
                        window = X_pad[n, :, h0:h0+FH, w0:w0+FW]
                        dW[f] += dA[n, f, oh, ow] * window
                        dX_pad[n, :, h0:h0+FH, w0:w0+FW] += dA[n, f, oh, ow] * self.W[f]
                dB[f] += np.sum(dA[n, f])

        dX = dX_pad[:, :, P:-P, P:-P] if P > 0 else dX_pad
        self.dW = dW / N
        self.dB = dB / N
        self.optimizer.update(self)
        return dX

class MaxPool2D:
    def __init__(self, pool_size, stride=None):
        self.pool_size = (pool_size, pool_size) if isinstance(pool_size, int) else pool_size
        self.stride = self.pool_size if stride is None else (stride, stride)

    def forward(self, X):
        self.X = X
        N, C, H, W = X.shape
        PH, PW = self.pool_size
        SH, SW = self.stride
        OH = (H - PH) // SH + 1
        OW = (W - PW) // SW + 1
        out = np.zeros((N, C, OH, OW))
        self.arg_max = np.zeros((N, C, OH, OW, 2), dtype=int)

        for n in range(N):
            for c in range(C):
                for oh in range(OH):
                    for ow in range(OW):
                        h0 = oh * SH
                        w0 = ow * SW
                        pool = X[n, c, h0:h0+PH, w0:w0+PW]
                        out[n, c, oh, ow] = np.max(pool)
                        idx = np.unravel_index(np.argmax(pool), pool.shape)
                        self.arg_max[n, c, oh, ow] = (h0 + idx[0], w0 + idx[1])
        return out

    def backward(self, dout):
        dX = np.zeros_like(self.X)
        N, C, OH, OW = dout.shape
        for n in range(N):
            for c in range(C):
                for oh in range(OH):
                    for ow in range(OW):
                        h_idx, w_idx = self.arg_max[n, c, oh, ow]
                        dX[n, c, h_idx, w_idx] += dout[n, c, oh, ow]
        return dX

# --- CNN Classifier ---
class Scratch2dCNNClassifier():
    def __init__(self, CNN, NN, n_epoch=1, batch_size=32, verbose=False):
        self.CNN = CNN
        self.NN = NN
        self.n_epoch = n_epoch
        self.batch_size = batch_size
        self.verbose = verbose
        self.log_loss = np.zeros(self.n_epoch)
        self.log_acc = np.zeros(self.n_epoch)

    def loss_function(self, y, yt):
        delta = 1e-7
        return -np.mean(yt * np.log(y + delta))

    def accuracy(self, Z, Y):
        return accuracy_score(Y, np.argmax(Z, axis=1))

    def fit(self, X, y, X_val=None, y_val=None):
        n_batches = int(np.ceil(X.shape[0] / self.batch_size))
        for epoch in range(self.n_epoch):
            get_mini_batch = GetMiniBatch(X, y, batch_size=self.batch_size)
            self.loss = 0
            for mini_X, mini_y in get_mini_batch:
                forward_data = mini_X[:, np.newaxis, :, :]
                for layer in self.CNN.values():
                    forward_data = layer.forward(forward_data)

                flatten = Flatten()
                forward_data = flatten.forward(forward_data)

                for layer in self.NN.values():
                    forward_data = layer.forward(forward_data)

                Z = forward_data
                dout = (Z - mini_y) / self.batch_size
                for layer in reversed(self.NN.values()):
                    dout = layer.backward(dout)

                dout = flatten.backward(dout)
                for layer in reversed(self.CNN.values()):
                    dout = layer.backward(dout)

                self.loss += self.loss_function(Z, mini_y)

            avg_loss = self.loss / n_batches
            y_pred_train = self.predict(X)
            train_accuracy = self.accuracy(y_pred_train, np.argmax(y, axis=1))
            self.log_loss[epoch] = avg_loss
            self.log_acc[epoch] = train_accuracy

            if self.verbose:
                print(f"Epoch {epoch+1}/{self.n_epoch}, Loss: {avg_loss:.4f}, Accuracy: {train_accuracy:.4f}")

            if X_val is not None and y_val is not None:
                y_pred_val = self.predict(X_val)
                val_accuracy = self.accuracy(y_pred_val, np.argmax(y_val, axis=1))
                print(f"Epoch {epoch+1}/{self.n_epoch}, Validation Accuracy: {val_accuracy:.4f}")

    def predict(self, X):
        pred_data = X[:, np.newaxis, :, :]
        for layer in self.CNN.values():
            pred_data = layer.forward(pred_data)
        flatten = Flatten()
        pred_data = flatten.forward(pred_data)
        for layer in self.NN.values():
            pred_data = layer.forward(pred_data)
        return pred_data

####
if __name__ == '__main__':
    mnist = fetch_openml('mnist_784', version=1, cache=True, as_frame=False)
    X, y = mnist['data'], mnist['target'].astype(int)

    num_samples = 1000
    X = X[:num_samples]
    y = y[:num_samples]
    X = X / 255.0
    X = X.reshape(-1, 28, 28)

    enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    y_onehot = enc.fit_transform(y[:, np.newaxis])
    X_train, X_val, y_train, y_val = train_test_split(X, y_onehot, test_size=0.2, random_state=42)

    initializer = HeInitializer()
    optimizer = SGD(lr=0.01)

    CNN = {
        'conv1': Conv2d(F=6, C=1, FH=5, FW=5, P=2, S=1, initializer=initializer, optimizer=optimizer, activation=ReLU()),
        'pool1': MaxPool2D(pool_size=2, stride=2)
    }

    NN = {
        'fc1': FC(n_in=6 * 14 * 14, n_out=100, initializer=initializer, optimizer=optimizer, activation=ReLU()),
        'fc2': FC(n_in=100, n_out=10, initializer=initializer, optimizer=optimizer, activation=Softmax())
    }

    model = Scratch2dCNNClassifier(CNN=CNN, NN=NN, n_epoch=1, batch_size=64, verbose=True)
    model.fit(X_train, y_train, X_val, y_val)


In [ ]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score

#####
class GetMiniBatch:
    def __init__(self, X, y, batch_size=20, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self._X = X[shuffle_index]
        self._y = y[shuffle_index]
        self._stop = (X.shape[0] + self.batch_size - 1) // self.batch_size

    def __len__(self):
        return self._stop

    def __getitem__(self, item):
        if item >= self._stop:
            raise IndexError("Mini-batch index out of range")
        p0 = item * self.batch_size
        p1 = min(p0 + self.batch_size, len(self._X))
        return self._X[p0:p1], self._y[p0:p1]

class ReLU:
    def forward(self, A):
        self.A = A
        return np.maximum(0, A)

    def backward(self, dZ):
        return dZ * (self.A > 0)

class Softmax:
    def forward(self, X):
        self.Z = np.exp(X - np.max(X, axis=1, keepdims=True))
        return self.Z / np.sum(self.Z, axis=1, keepdims=True)

    def backward(self, dZ):
        dA = self.Z * dZ
        s = np.sum(dA, axis=1, keepdims=True)
        dA -= self.Z * s
        return dA

class FC:
    def __init__(self, n_in, n_out, initializer, optimizer, activation):
        self.W = initializer.W(n_in, n_out)
        self.B = initializer.B(n_out)
        self.optimizer = optimizer
        self.activation = activation

    def forward(self, X):
        self.X = X
        self.Z = np.dot(X, self.W) + self.B
        return self.activation.forward(self.Z)

    def backward(self, dA):
        dZ = self.activation.backward(dA)
        self.dW = np.dot(self.X.T, dZ)
        self.dB = np.sum(dZ, axis=0)
        dX = np.dot(dZ, self.W.T)
        self.optimizer.update(self)
        return dX

class SGD:
    def __init__(self, lr):
        self.lr = lr

    def update(self, layer):
        layer.W -= self.lr * layer.dW
        layer.B -= self.lr * layer.dB

class HeInitializer:
    def W(self, *args):
        if len(args) == 2:
            n_in, n_out = args
            return np.random.randn(n_in, n_out) * np.sqrt(2 / n_in)
        elif len(args) == 4:
            F, C, FH, FW = args
            return np.random.randn(F, C, FH, FW) * np.sqrt(2 / (C * FH * FW))
        else:
            raise ValueError("Invalid shape")

    def B(self, n_out):
        return np.zeros(n_out)

class Flatten:
    def __init__(self):
        self.shape = None

    def forward(self, X):
        self.shape = X.shape
        return X.reshape(len(X), -1)

    def backward(self, dout):
        return dout.reshape(self.shape)

class Conv2d:
    def __init__(self, F, C, FH, FW, P, S, initializer=None, optimizer=None, activation=None):
        self.F, self.C, self.FH, self.FW, self.P, self.S = F, C, FH, FW, P, S
        self.initializer = initializer
        self.optimizer = optimizer
        self.activation = activation
        self.W = initializer.W(F, C, FH, FW)
        self.B = initializer.B(F)

    def output_shape2d(self, H, W):
        OH = (H + 2 * self.P - self.FH) // self.S + 1
        OW = (W + 2 * self.P - self.FW) // self.S + 1
        return OH, OW

    def forward(self, X):
        self.X = X
        N, C, H, W = X.shape
        OH, OW = self.output_shape2d(H, W)
        A = np.zeros((N, self.F, OH, OW))
        X_pad = np.pad(X, ((0, 0), (0, 0), (self.P, self.P), (self.P, self.P)), 'constant')

        for n in range(N):
            for f in range(self.F):
                for oh in range(OH):
                    for ow in range(OW):
                        h0 = oh * self.S
                        w0 = ow * self.S
                        window = X_pad[n, :, h0:h0+self.FH, w0:w0+self.FW]
                        A[n, f, oh, ow] = np.sum(window * self.W[f]) + self.B[f]

        return self.activation.forward(A) if self.activation else A

    def backward(self, dA):
        if self.activation:
            dA = self.activation.backward(dA)

        N, F, OH, OW = dA.shape
        _, _, H, W = self.X.shape
        FH, FW, P, S = self.FH, self.FW, self.P, self.S
        dX = np.zeros_like(self.X)
        dW = np.zeros_like(self.W)
        dB = np.zeros_like(self.B)

        X_pad = np.pad(self.X, ((0,0), (0,0), (P,P), (P,P)), 'constant')
        dX_pad = np.pad(dX, ((0,0), (0,0), (P,P), (P,P)), 'constant')

        for n in range(N):
            for f in range(F):
                for oh in range(OH):
                    for ow in range(OW):
                        h0 = oh * S
                        w0 = ow * S
                        window = X_pad[n, :, h0:h0+FH, w0:w0+FW]
                        dW[f] += dA[n, f, oh, ow] * window
                        dX_pad[n, :, h0:h0+FH, w0:w0+FW] += dA[n, f, oh, ow] * self.W[f]
                dB[f] += np.sum(dA[n, f])

        dX = dX_pad[:, :, P:-P, P:-P] if P > 0 else dX_pad
        self.dW = dW / N
        self.dB = dB / N
        self.optimizer.update(self)
        return dX

class MaxPool2D:
    def __init__(self, pool_size, stride=None):
        self.pool_size = (pool_size, pool_size) if isinstance(pool_size, int) else pool_size
        self.stride = self.pool_size if stride is None else (stride, stride)

    def forward(self, X):
        self.X = X
        N, C, H, W = X.shape
        PH, PW = self.pool_size
        SH, SW = self.stride
        OH = (H - PH) // SH + 1
        OW = (W - PW) // SW + 1
        out = np.zeros((N, C, OH, OW))
        self.arg_max = np.zeros((N, C, OH, OW, 2), dtype=int)

        for n in range(N):
            for c in range(C):
                for oh in range(OH):
                    for ow in range(OW):
                        h0 = oh * SH
                        w0 = ow * SW
                        pool = X[n, c, h0:h0+PH, w0:w0+PW]
                        out[n, c, oh, ow] = np.max(pool)
                        idx = np.unravel_index(np.argmax(pool), pool.shape)
                        self.arg_max[n, c, oh, ow] = (h0 + idx[0], w0 + idx[1])
        return out

    def backward(self, dout):
        dX = np.zeros_like(self.X)
        N, C, OH, OW = dout.shape
        for n in range(N):
            for c in range(C):
                for oh in range(OH):
                    for ow in range(OW):
                        h_idx, w_idx = self.arg_max[n, c, oh, ow]
                        dX[n, c, h_idx, w_idx] += dout[n, c, oh, ow]
        return dX

class Flatten:
    def __init__(self):
        self.shape = None

    def forward(self, X):
        self.shape = X.shape
        return X.reshape(len(X), -1)

    def backward(self, dout):
        return dout.reshape(self.shape)

###
class LeNetCNNClassifier():
    def __init__(self, CNN, NN, n_epoch=1, batch_size=32, verbose=False):
        self.CNN = CNN
        self.NN = NN
        self.n_epoch = n_epoch
        self.batch_size = batch_size
        self.verbose = verbose
        self.log_loss = np.zeros(self.n_epoch)
        self.log_acc = np.zeros(self.n_epoch)

    def loss_function(self, y, yt):
        delta = 1e-7
        return -np.mean(yt * np.log(y + delta))

    def accuracy(self, Z, Y):
        return accuracy_score(Y, np.argmax(Z, axis=1))

    def fit(self, X, y, X_val=None, y_val=None):
        n_batches = int(np.ceil(X.shape[0] / self.batch_size))
        for epoch in range(self.n_epoch):
            get_mini_batch = GetMiniBatch(X, y, batch_size=self.batch_size)
            self.loss = 0
            for mini_X, mini_y in get_mini_batch:
                forward_data = mini_X[:, np.newaxis, :, :]
                for layer in self.CNN.values():
                    forward_data = layer.forward(forward_data)

                flatten = Flatten()
                forward_data = flatten.forward(forward_data)

                for layer in self.NN.values():
                    forward_data = layer.forward(forward_data)

                Z = forward_data
                dout = (Z - mini_y) / self.batch_size
                for layer in reversed(self.NN.values()):
                    dout = layer.backward(dout)

                dout = flatten.backward(dout)
                for layer in reversed(self.CNN.values()):
                    dout = layer.backward(dout)

                self.loss += self.loss_function(Z, mini_y)

            avg_loss = self.loss / n_batches
            y_pred_train = self.predict(X)
            train_accuracy = self.accuracy(y_pred_train, np.argmax(y, axis=1))
            self.log_loss[epoch] = avg_loss
            self.log_acc[epoch] = train_accuracy

            if self.verbose:
                print(f"Epoch {epoch+1}/{self.n_epoch}, Loss: {avg_loss:.4f}, Accuracy: {train_accuracy:.4f}")

            if X_val is not None and y_val is not None:
                y_pred_val = self.predict(X_val)
                val_accuracy = self.accuracy(y_pred_val, np.argmax(y_val, axis=1))
                print(f"Epoch {epoch+1}/{self.n_epoch}, Validation Accuracy: {val_accuracy:.4f}")

    def predict(self, X):
        pred_data = X[:, np.newaxis, :, :]
        for layer in self.CNN.values():
            pred_data = layer.forward(pred_data)
        flatten = Flatten()
        pred_data = flatten.forward(pred_data)
        for layer in self.NN.values():
            pred_data = layer.forward(pred_data)
        return pred_data

#
if __name__ == '__main__':
    #MNIST data
    mnist = fetch_openml('mnist_784', version=1, cache=True, as_frame=False)
    X, y = mnist['data'], mnist['target'].astype(int)

    ####
    num_samples = 1000
    X = X[:num_samples]
    y = y[:num_samples]

    ###
    X = X / 255.0
    X = X.reshape(-1, 28, 28)
    enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    y_onehot = enc.fit_transform(y[:, np.newaxis])
    X_train, X_val, y_train, y_val = train_test_split(X, y_onehot, test_size=0.2, random_state=42)

    #LeNet architecture
    initializer = HeInitializer()
    optimizer = SGD(lr=0.01)

    CNN_lenet = {
        'conv1': Conv2d(F=6, C=1, FH=5, FW=5, P=2, S=1, initializer=initializer, optimizer=optimizer, activation=ReLU()),
        'pool1': MaxPool2D(pool_size=2, stride=2),
        'conv2': Conv2d(F=16, C=6, FH=5, FW=5, P=0, S=1, initializer=initializer, optimizer=optimizer, activation=ReLU()),
        'pool2': MaxPool2D(pool_size=2, stride=2)
    }

    #
    dummy_input = np.zeros((1, 1, 28, 28))
    output_cnn = dummy_input
    for layer in CNN_lenet.values():
        output_cnn = layer.forward(output_cnn)
    flatten_size = np.prod(output_cnn.shape[1:])

    NN_lenet = {
        'fc1': FC(n_in=flatten_size, n_out=120, initializer=initializer, optimizer=optimizer, activation=ReLU()),
        'fc2': FC(n_in=120, n_out=84, initializer=initializer, optimizer=optimizer, activation=ReLU()),
        'fc3': FC(n_in=84, n_out=10, initializer=initializer, optimizer=optimizer, activation=Softmax())
    }

    #
    lenet_model = LeNetCNNClassifier(CNN=CNN_lenet, NN=NN_lenet, n_epoch=1, batch_size=64, verbose=True)
    lenet_model.fit(X_train, y_train, X_val, y_val)

    #
    y_pred_val_lenet = lenet_model.predict(X_val)
    accuracy_lenet = lenet_model.accuracy(y_pred_val_lenet, np.argmax(y_val, axis=1))
    print(f"\nLeNet Validation Accuracy (1000 samples, 1 epoch): {accuracy_lenet:.4f}")

Both AlexNet and VGG16 are historically significant Convolutional Neural Network (CNN) architectures that played crucial roles in the advancement of deep learning for image recognition. They demonstrated the power of deep CNNs in achieving state-of-the-art results on large-scale image classification tasks, particularly in the ImageNet Large Scale Visual Recognition Challenge (ILSVRC).

AlexNet (2012):

Key Contributions:

AlexNet is widely considered to be the model that ignited the deep learning revolution in computer vision. It achieved a breakthrough performance in the ILSVRC 2012 competition, significantly outperforming previous methods.

Architecture: 
It consists of 8 layers: 5 convolutional layers and 3 fully connected layers. Notably, it was one of the first CNNs to utilize:

ReLU (Rectified Linear Unit) activation function: This non-saturating activation function accelerated training compared to traditional sigmoid and tanh functions.

Multiple GPUs: The model was trained on two GPUs in parallel, which was crucial for handling the computational demands of the deeper network and the large ImageNet dataset.

Local Response Normalization (LRN): A normalization technique that helped in generalization.

Overlapping Pooling: Max pooling layers with overlapping windows, which slightly reduced the error rate.

Dropout: A regularization technique applied to the fully connected layers to prevent overfitting.

Data Augmentation: Techniques like random cropping, scaling, and horizontal flipping were used to increase the size of the training data and improve the model's robustness.

Impact:

AlexNet's success demonstrated the effectiveness of deep CNNs for complex image recognition tasks and inspired a surge of research and development in the field.

VGG16 (2014):

Key Contributions:

Developed by the Visual Geometry Group (VGG) at the University of Oxford, VGG16 (and its variant VGG19) emphasized the importance of network depth. It achieved excellent performance in the ILSVRC 2014 competition.

Architecture: 

VGG16 is characterized by its very deep architecture, consisting of 16 layers with learnable weights (13 convolutional layers and 3 fully connected layers). Its main architectural characteristics include:

Small Convolutional Filters: It exclusively uses 3x3 convolutional filters throughout the network. Multiple stacked 3x3 convolutions have the same receptive field as larger filters but with more non-linearities and fewer parameters.

Max Pooling: 2x2 max pooling layers with a stride of 2 are used for downsampling.

Uniformity: The architecture has a very uniform structure, making it relatively easy to understand and implement.

Large Number of Parameters: Due to its depth and fully connected layers with a large number of neurons, VGG16 has a significantly larger number of parameters compared to AlexNet.

Impact: 

VGG16 demonstrated that increasing the depth of a CNN can lead to significant improvements in performance. Its simple and effective architecture became a popular choice for many image recognition tasks and served as a foundation for subsequent, even deeper networks. It also became a widely used feature extractor for transfer learning.

AlexNet was a pioneering work that brought deep CNNs to the forefront of image recognition, while VGG16 further solidified the importance of depth in CNN architectures through its simple yet effective design. Both models are considered foundational in the history of deep learning for computer vision and have paved the way for more advanced and efficient architectures.

Convolutional Layer Calculations


Formulae:

Output Height ($O_H$): $\lfloor \frac{I_H - F_H + 2P}{S} \rfloor + 1$

Output Width ($O_W$): $\lfloor \frac{I_W - F_W + 2P}{S} \rfloor + 1$

Number of Parameters: $(F_H \times F_W \times C_{in} + 1) \times C_{out}$

    $F_H$: Filter Height

    $F_W$: Filter Width

    $C_{in}$: Number of Input Channels

    $P$: Padding

    $S$: Stride

    $C_{out}$: Number of Output Channels (number of filters)

    The '+ 1' accounts for the bias term for each filter.

Layer 1:

Input size: 144 x 144 x 3

Filter size: 3 x 3 x 6

Stride: 1

Padding: none

Output size: 142 x 142 x 6

Number of Parameters: (3 x 3 x 3 + 1) x 6 = 168

Layer 2:

Input size: 60 x 60 x 24

Filter size: 3 x 3 x 48

Stride: 1

Padding: none

Output size: 58 x 58 x 48

Number of Parameters: (3 x 3 x 24 + 1) x 48 = 10416

Layer 3:

Input size: 20 x 20 x 10

Filter size: 3 x 3 x 20

Stride: 2

Padding: none

Output size: 9 x 9 x 20

Number of Parameters: (3 x 3 x 10 + 1) x 20 = 1820

for Layer 3:

The convolution in Layer 3 doesn't perfectly cover the input boundaries due to the stride of 2 and no padding. This can lead to some edge information being missed.


Why 3x3 Filters Are Commonly Used Instead of Larger Ones Such as 7x7


Computational Efficiency: Larger filters (e.g., 7x7) require significantly more computations than smaller ones. For a single convolution operation, a 7x7 filter involves 49 multiplications per location, whereas a 3x3 filter only requires 9. This difference becomes substantial in deep networks with many convolutional layers, impacting training and inference speed.

Deeper Networks and Increased Non-linearity: Stacking multiple 3x3 filters can achieve the same effective receptive field as a larger filter but with greater depth. For instance, two 3x3 filters stacked together have a receptive field of 5x5, and three 3x3 filters have a receptive field of 7x7. Deeper networks can learn more complex and hierarchical representations of the input data. Additionally, each convolutional layer is typically followed by a non-linear activation function (like ReLU). Stacking multiple 3x3 filters introduces more non-linearities, enabling the network to learn more intricate features.

Fewer Parameters: Using multiple 3x3 filters instead of a single large filter reduces the number of parameters in the network. This helps to mitigate overfitting, especially when dealing with limited training data. For example, consider a convolutional layer with C input channels and C output channels. A single 7x7 filter would require C x 7 x 7 x C parameters, while two stacked 3x3 filters would require 2 x C x 3 x 3 x C parameters. It can be easily shown that the latter has fewer parameters.

In essence, 3x3 filters offer a good balance between computational efficiency, representational power, and parameter efficiency.

The Effect of a 1x1 Filter with No Height or Width


Channel-wise Transformation: A 1x1 filter operates across the depth (channel) dimension of the input. It performs a linear transformation of the input channels, allowing the network to increase or decrease the number of channels.

Dimensionality Reduction: 1x1 convolutions can be used to reduce the number of channels, which can help to decrease the computational cost of subsequent layers. For example, if an input has 256 channels, a 1x1 convolution can reduce it to 64 channels before feeding it into a computationally expensive 3x3 convolution. This is a key technique used in architectures like Inception and ResNet.

Increased Non-linearity: When used in conjunction with non-linear activation functions, 1x1 convolutions can introduce additional non-linearities into the network. This can enhance the network's ability to learn complex functions.

1x1 filters, despite their simplicity, are powerful tools for manipulating the channel dimension, reducing computational cost, and increasing non-linearity in CNNs.